In [ ]:
# old patients simulation

import pandas as pd
import os

# === Step 1: Set up paths ===

# Get absolute path to this script's folder
try:
    # Use __file__ if available
    project_root = os.path.dirname(os.path.abspath(__file__))
except NameError:
    # Fallback to the current working directory
    project_root = os.getcwd()

# point to the parent directory (ai_service)
project_root = os.path.dirname(project_root)

# Path to dataset
dataset_path = os.path.join(project_root, "data", "Dataset.csv")

# Debugging: Print the resolved path
print("Looking for dataset at:", dataset_path)

# Path to save extracted patient data (new folder)
output_dir = os.path.join(project_root, "data", "patients")
os.makedirs(output_dir, exist_ok=True)

# === Step 2: Read and clean dataset ===

# Read raw CSV lines
try:
    with open(dataset_path, "r", encoding="utf-8") as f:
        lines = f.readlines()
except FileNotFoundError:
    print(f"Error: The file '{dataset_path}' was not found. Please check the path.")
    exit(1)

# Clean header and data lines
header = lines[0].replace('"', '').strip().split(",")
data = [line.replace('"', '').strip().split(",") for line in lines[1:]]

# Build DataFrame
df = pd.DataFrame(data, columns=header)

# Convert all numeric columns (except ID)
for col in df.columns:
    if col != "seqn":
        df[col] = pd.to_numeric(df[col], errors="coerce")

# === Step 3: Extract patient row and save it ===

# Get patient row number
patient_1 = df.iloc[[6000]]

# Save to CSV
patient_path = os.path.join(output_dir, "patient_1_cleaned.csv")
patient_1.to_csv(patient_path, index=False)


print("✅ Saved clean patient_1_cleaned.csv to:", patient_path)

Looking for dataset at: c:\Users\dell\OneDrive\Desktop\Final-Project-NABDH\ai_service\data\Dataset.csv
✅ Saved clean patient_1_cleaned.csv to: c:\Users\dell\OneDrive\Desktop\Final-Project-NABDH\ai_service\data\patients\patient_1_cleaned.csv


In [ ]:
import pandas as pd
import os

# Go up one level from 'notebooks' to project root
base_dir = os.path.dirname(os.getcwd())  # get out of 'notebooks'
patient_path = os.path.join(base_dir, "data", "patients", "patient_1_cleaned.csv")

# === 1. Load the patient CSV
patient = pd.read_csv(patient_path)

print("✅ Loaded:", patient_path)

# === 2. Define normal ranges (copy-paste from Apriori general script) ===
normal_ranges = {
    'lbxsal': (135, 145), 'lbdsalsi': (135, 145), 'lbxsassi': (10, 40), 'lbxsapsi': (44, 147),
    'lbxsbu': (7, 20), 'lbdsbusi': (2.5, 7.1), 'lbxsca': (8.5, 10.2), 'lbdscasi': (2.12, 2.55),
    'lbxsck': (22, 198), 'lbxsch': (0, 200), 'lbdschsi': (0, 5.17), 'lbxsc3si': (22, 29),
    'lbxscr': (0.74, 1.35), 'lbdscrsi': (65.4, 119.3), 'lbxsgtsi': (9, 48), 'lbxsgl': (70, 99),
    'lbdsglsi': (3.9, 5.5), 'lbxsir': (60, 170), 'lbdsirsi': (10.7, 30.4), 'lbxsldsi': (140, 280),
    'lbxsph': (2.5, 4.5), 'lbdsphsi': (0.81, 1.45), 'lbxstb': (0.1, 1.2), 'lbdstbsi': (1.71, 20.5),
    'lbxstp': (6.0, 8.3), 'lbdstpsi': (60, 83), 'lbxsua': (3.5, 7.2), 'lbdsuasi': (208, 428),
    'lbxsnasi': (135, 145), 'lbxsksi': (3.5, 5.1), 'lbxsclsi': (98, 107), 'lbxsossi': (275, 295),
    'lbxsgb': (2.0, 3.5), 'lbdsgbsi': (20, 35), 'lbxstr': (0, 150), 'lbdstrsi': (0, 1.7),
    'lbxsatsi': (20, 55)
}

# === 3. Our Normalize function ===
def normalize(val, low, high):
    if val < low:
        return -1
    elif val > high:
        return 1
    return 0

# === 4. Analyze abnormal markers ===
# up arrow for 1 and updown for -1
abnormal_markers = []

for col, (low, high) in normal_ranges.items():
    val = pd.to_numeric(patient[col].values[0], errors="coerce")
    status = normalize(val, low, high)

    if status == -1:
        abnormal_markers.append(f"{col}↓")
    elif status == 1:
        abnormal_markers.append(f"{col}↑")

# === 5. Show the result ===
print("🧬 Abnormal markers for this patient '''Patient1''':\n")
for marker in abnormal_markers:
    print("•", marker)


✅ Loaded: c:\Users\dell\OneDrive\Desktop\Final-Project-NABDH\ai_service\data\patients\patient_1_cleaned.csv
🧬 Abnormal markers for this patient '''Patient1''':

• lbxsal↓
• lbdsalsi↓
• lbxsapsi↓
• lbxsldsi↓
• lbxsclsi↑


In [ ]:
# matching the extracted abnormal marks with general rules

import pandas as pd

# 1. Load abnormal markers
patient_markers = abnormal_markers

# 2. Load generated Apriori rules
with open("generated_rules.txt", "r", encoding="utf-8") as f:
    rules = f.readlines()

# 3. Filter rules that only match patient markers
matching_rules = []

for rule in rules:
    rule = rule.strip()
    if not rule:
        continue

    # Extract antecedents from rule
    if "If [" in rule and "] →" in rule:
        ant_section = rule.split("If [")[1].split("]")[0]
        antecedents = [x.strip() for x in ant_section.split(",")]

        # Check if any antecedents match patient markers
        if any(a in patient_markers for a in antecedents):
            matching_rules.append(rule)

# 4. Display results
print("\n📌 Matching Rules for This Patient:\n")
for r in matching_rules:
    print("•", r)

# 5. Optional: Save matching rules to a file
with open("matched_rules_for_patient_1.txt", "w", encoding="utf-8") as f:
    for r in matching_rules:
        f.write(r + "\n")

print("\n✅ Matching rules saved in 'matched_rules_for_patient_1.txt'")



📌 Matching Rules for This Patient:


✅ Matching rules saved in 'matched_rules_for_patient_1.txt'
